In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max
from PIL import Image

from skimage.morphology import skeletonize, thin
from skimage.util import invert
from skimage import measure
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk
from skimage import feature
from skimage.filters import roberts, sobel, scharr, prewitt
import seaborn as sns
sns.reset_orig()

from skimage import restoration

from skimage import exposure
from skimage.filters import gaussian

from scipy import ndimage

from time import time

from skimage.color import rgb2gray
from scipy import ndimage as ndi

from skimage.filters.rank import entropy
import scipy


import networkx as nx
from scipy import interpolate

from skimage.segmentation import random_walker
from skimage.data import binary_blobs
from skimage.exposure import rescale_intensity
import glob

from skimage.morphology import disk
from skimage import io


from skimage.external import tifffile

import pandas as pd

from sklearn.cluster import KMeans

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

import threading



/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [7]:
csv = pd.read_csv('all2.csv')
lab=io.imread('labeled_green_cells.tif')

In [8]:
data = np.asarray(csv.values)
data2 = data[:,1:4]
data2= data2.astype(int)
data2

array([[   3, 1420, 1837],
       [   6, 1415, 1850],
       [   9, 1499, 1845],
       ...,
       [ 543,  902,  769],
       [ 540, 1151, 1342],
       [ 585, 1305,  799]])

In [9]:
pts = []
for i in range(len(data2)):
    print(i)
    pts.append(np.where((lab==data[i, 5]) ))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
pts2 = []
for p in pts:
    a1 = []
    for p2 in p:
        a2 = np.asarray(p2)
        a1.append(a2)
        print(np.asarray(p2) )
    
    
    pts2.append(np.asarray(a1) )

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]
[1416 1416 1416 1417 1417 1417 1418 1418 1418 1418 1419 1419 1419 1419
 1419 1420 1420 1420 1420 1420 1420 1420 1421 1421 1421 1421 1421 1421
 1421 1422 1415 1416 1416 1416 1416 1416 1417 1417 1417 1417 1417 1417
 1418 1418 1418 1418 1418 1418 1418 1418 1418 1419 1419 1419 1419 1419
 1419 1419 1419 1419 1420 1420 1420 1420 1420 1420 1420 1420 1420 1421
 1421 1421 1421 1421 1421 1422 1422 1422 1422 1422]
[1835 1836 1837 1835 1836 1838 1835 1836 1837 1838 1834 1836 1837 1838
 1839 1834 1835 1836 1837 1838 1839 1840 1834 1835 1836 1837 1838 1839
 1840 1838 1836 1835 1836 1837 1838 1840 1833 1834 1835 1838 1839 1840
 1832 1833 1834 1835 1836 1837 1838 1839 1840 1832 1833 1834 1835 1836
 1837 1838 1839 1840 1831 1832 1833 1834 1835 1836 1837 1838 1839 1832
 1833 1834 1835 1836 1838 1832 1834 1835 1836 1837]
[1 1 1 1 1 1 1 1 1 1 1

 1192 1193 1192 1191 1192 1193 1191 1192 1193]
[92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92
 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92
 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92 92
 92 92 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93
 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93
 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93
 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93 93
 93 93 93 93 93 93 93 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94
 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94
 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94
 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 94 95 95 95
 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95
 95 95 95 95 95 95 95 95]
[579 579 580 580 581 581 581 582 582 582 582 582 58

In [13]:
pts[0]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1416, 1416, 1416, 1417, 1417, 1417, 1418, 1418, 1418, 1418, 1419,
        1419, 1419, 1419, 1419, 1420, 1420, 1420, 1420, 1420, 1420, 1420,
        1421, 1421, 1421, 1421, 1421, 1421, 1421, 1422, 1415, 1416, 1416,
        1416, 1416, 1416, 1417, 1417, 1417, 1417, 1417, 1417, 1418, 1418,
        1418, 1418, 1418, 1418, 1418, 1418, 1418, 1419, 1419, 1419, 1419,
        1419, 1419, 1419, 1419, 1419, 1420, 1420, 1420, 1420, 1420, 1420,
        1420, 1420, 1420, 1421, 1421, 1421, 1421, 1421, 1421, 1422, 1422,
        1422, 1422, 1422]),
 array([1835, 1836, 1837, 1835, 1836, 1838, 1835, 1836, 1837, 1838, 1834,
        1836, 1837, 1838, 1839, 1834, 1835, 1836, 1837, 1838, 1839, 1840,
        1834, 1835, 1836, 1837, 

In [19]:
pts2[0][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [22]:
pts2[0]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1],
       [1416, 1416, 1416, 1417, 1417, 1417, 1418, 1418, 1418, 1418, 1419,
        1419, 1419, 1419, 1419, 1420, 1420, 1420, 1420, 1420, 1420, 1420,
        1421, 1421, 1421, 1421, 1421, 1421, 1421, 1422, 1415, 1416, 1416,
        1416, 1416, 1416, 1417, 1417, 1417, 1417, 1417, 1417, 1418, 1418,
        1418, 1418, 1418, 1418, 1418, 1418, 1418, 1419, 1419, 1419, 1419,
        1419, 1419, 1419, 1419, 1419, 1420, 1420, 1420, 1420, 1420, 1420,
        142

In [28]:
for i in range(len(data)):
    print(data[i][5])

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.

In [34]:
pts3 = []
for i in range(len(pts2)):
    
    
    for x in range(len(pts2[i][0])):
        
        datapoints = [pts2[i][0, x], pts2[i][1, x], pts2[i][2, x], data[i][5]]
        pts3.append(datapoints)
    

In [36]:
len(pts3)

39361

In [38]:
np.savetxt("points.csv", pts3, delimiter=",")

In [5]:
len(data)

578

In [25]:
file1 = open("points.txt","a")


for i in range(len(data2)):
    file1.write(str(pts[i])+'\n'+'\n')

file1.close() 

In [14]:
clustering = DBSCAN(eps=100, min_samples=1).fit(data2)
labels = clustering.labels_+1
print(labels)


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  2  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  3  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  3  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  4  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  5  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  4  1  1  1  1  1  1  1  1  1  1  1  6  1
  1  1  1  1  1  1  1  1  1  1  1  1  7  1  1  1  7  1  1  1  1  1  1  1
  1  1  8  1  1  1  1  1  1  1  1  9  1  1  1 10  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 11 12  1  1  1 11  1  1 13  1 14  1  1  1  1 12  1  1
  1  1  1 10  1  1 10 10 10 15 10 10 10 10  1  1 16 16  1 10  1  1 10 17
 10 10 10 10 15  1 15 13 10 10 10 10  1 10 10 10  1  1 10 10 10 10 10 15
 10 18 10 10 10 10 19 19 11 11 10 20 13 19 21  1  1

In [15]:
data[0]

array([0.00000000e+00, 3.00000000e+00, 1.42000000e+03, 1.83700000e+03,
       2.40000000e+02, 1.00000000e+00, 1.00000000e+00, 1.83100000e+03,
       1.41800000e+03, 5.86929297e+02, 0.00000000e+00, 1.81000000e+03,
       1.32100000e+03, 6.46811410e+02])

In [ ]:
ret = np.zeros(lab.shape, dtype='uint32')
for i in range(len(labels)):
    adddd = (lab==data[i, 5])*labels[i]
    
    
    print(i, labels[i])
    ret =ret + adddd

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1


In [ ]:
INTERVAL = 6 

ret = np.zeros(lab.shape, dtype='uint32')
for i in range(len(labels)):
    
    
    min_sub = INTERVAL
    max_add = INTERVAL
    
    
    
    if data2[i, 0]<INTERVAL:
        min_sub = i
        
    if data2[i, 0]+INTERVAL>len(lab):
        max_add= len(lab)-1
    
    
    
    adddd = (lab[ data2[i,0]-min_sub: data2[i,0]+max_add ] ==data[i, -1])*labels[i]
    
    
    print(np.unique(adddd), labels[i])
    ret[ data2[i,0]-min_sub: data2[i,0]+max_add ] =ret[ data2[i,0]-min_sub, data2[i,0]+max_add ] + adddd

[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 2
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 3
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 3
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1
[0] 1


In [7]:
clusterunique = np.unique(ret)

In [8]:
clusterunique

array([0], dtype=uint32)

In [9]:
def label_cluster(labeled_arr, vals, cluster_labels, p1, p2, output):
    
    ret = np.zeros(labeled_arr.shape, dtype='uint32')
    
    for i in range(p1, p2):
    
        adddd = (labeled_arr==vals[i, -1])*cluster_labels[i]


        print(int(data[i, -1]) )
        ret =ret + adddd
    output = ret

In [ ]:
THREADS = 10
step = int(len(data2)/THREADS)

thread_arr = []
relabeledarr = []
for i in range((THREADS)):
    relabeledarr.append(np.zeros(lab.shape, dtype='uint32'))
    
    
    
    if i < THREADS-1:
        print(step*i, step*(i+1))
        
        
        td = threading.Thread(target=label_cluster, args=(lab,data, labels, step*i, step*(i+1)  , relabeledarr[i] ))
        
        thread_arr.append(td)
        
    else: 
        print(step*i, len(data2))
        td = threading.Thread(target=label_cluster, args=(lab,data, labels, step*i, len(data2)  , relabeledarr[i] )) 
        thread_arr.append(td)

        
        
        
for i in range((THREADS)):
    (thread_arr[i]).start()

for i in range((THREADS)):
    (thread_arr[i]).join()  

    
print("Done!") 
    #td = threading.Thread(target=label_cluster, args=(lab,data2, labels,  ,  , relabeledarr[0] )) 


0 57
57 114
114 171
171 228
228 285
285 342
342 399
399 456
456 513
513 578
17258286343400229
523115

1



457


344524
458401116230287

2
173
59






In [ ]:
tifffile.imsave('labelwithout0.tif', lab3)